# Working with Processes and Jobs
Unity helps teams move quickly from algorithm development and testing in Jupyter into large-scale processing with the Scaled Processing System (SPS). In order to do this, algorithms developed in Jupyter must be: 
1. registered in the Application Catalog (see 1_working_with_applications.ipynb) as _Applications_, 
2. deployed to an Application Deployment and Execution Service (ADES) in SPS where they are referred to as _Processes_, and then may be 
3. run on the SPS as _Jobs_.

The intent of this tutorial is to help familiarize yourself with execution of code at scale using _Processes_ and _Jobs_. Unity provides users the ability to execute _Jobs_ to produce data. The steps below will showcase how a job is typically submitted to the Unity Platform.

## 0. Set up imports and predefined variables

For this Tutorial we will make use of a wrapper API that is called api.py (in the utils folder). It helps to manage things like authentication.

In [ ]:
from utils import api
import json
from IPython.display import JSON
import requests
import time
from datetime import datetime

app_name = "job-l1b-cwl:develop"

## 1. List Processes

The Unity API defines a way of viewing all deployed application packages (a.k.a., Processes) on the system using the 'get_apps' function. After a successful deployment of an application package to the Processing endpoint, you should see a new entry for the deployed application.

The 'id' field in the returned JSON document is how you will make calls to execute the Process and see existing Jobs for a given Process.

In [ ]:
apps = api.get_apps()
for application in apps:
    print("Process ID: {}".format(application['id']))
    print("Process Version: {}".format(application['processVersion']))
    print("Process description: {}".format(application['abstract']))
    print("")
    
print("\n\nFull JSON Response")
JSON(apps)

## 2. Execute a job
Before deploying Applications and working with jobs, it is assumed that a system administrator has deployed an ADES. These are often called "venues" or "environments", and some examples may be dev, test, prod, etc. To run a Job, you need a Process available as well (a deployed Application). In this case we have a demo application deployed and referenced in the setup step 0 above.

With an ADES and a Process ready to accept our Job, we can submit a Job along with any input data or parameters that are needed (as defined by a template job definition for a particular Application). In this example case, none are needed so `inputs` is blank. The response will provide a Job ID that we will store in a variable called `job_id` for use later.

***NOTE*** - the sample application does not provide input parameters or output parameters at this time. These are coming in a future version of the job control endpoint. Future jobs will allow:
- Explicit inputs to be used (no magic)
  - inputs can be Unity Resources or DAAC resources
- Explicit output 'collection' to which the data will be written

In [ ]:
data = {
  "mode": "async",
  "response": "document",
  "inputs": [
  ],
  "outputs": [
    {
      "id": "output",
      "transmissionMode": "reference"
    }
  ]
}

try:

    # Store Job ID to use in future steps
    job_id = api.submit_job(app_name, data)

    # If the job submission is successful, print a success message along with the returned JOB-ID
    print("\nJob Submission Successful!\nJOB ID: {}\n".format(job_id))

except requests.exceptions.HTTPError as e:
    # An error has occurred, print the error message that was generated
    print(e.reason)

## 2. Get the Job status
The code below will demonstrate how one can check the status of the Job. The potential status responses are documented [here] _need a reference to process status_.

In this example, we will look up the status of the predfined application name from the setup step 0, and the job ID that was returned previously. This function will loop/block until the process ends. You will see a printout every 5 seconds.

In [ ]:
# Set jobID here if you'd like
# job_id = "76ecf5a6-54b7-4057-bf83-89524a0f709d"

try:
    
    job_status = api.get_job_status(app_name, job_id)

    while job_status == "running":
        print("Status for job \"{}\" ({}): {}".format(job_id, datetime.now().strftime("%H:%M:%S"), job_status))
        time.sleep(5)
        job_status = api.get_job_status(app_name, job_id)
    
    # Print the job status
    print("\nStatus for job \"{}\" ({}): {}\n".format(job_id, datetime.now().strftime("%H:%M:%S"),job_status))
    
except requests.exceptions.HTTPError as e:
    # An error has occurred, print the error message that was generated
    print(e.reason)


## 3. Get Job results
Now that we've monitored the status of a Job, and verified that is has completed, we can retreive information about where the generated data is located. 

In this example, we will use the predefined Process name, and the Job ID that was returned previously.

In [ ]:
# Set jobID here if you'd like
#job_id = "35284103-b7ce-4ea4-86d1-c0dd0622f898"

try:

    job_data = api.get_job_by_id(app_name, job_id)

    # for data in job_data:
    #     print("ID: " + data['id'])
    #     print("Type: " + data['mimeType'])
    #     print("Location: " + data['href'])
        
except requests.exceptions.HTTPError as e:
    print(e.reason)

print("\nFull JSON output data object:")
JSON(job_data)

## So... where are the results?

Currently the results are not made available to the endpoint, but a future release will include a STAC document of generated files and their locations. For now, the results are written to the `SNDR_SNPP_ATMS_L1B_OUTPUT___1` product type in the Unity system. Again, future releases will allow the specification of the output collection (by name or version).

## 4. List the Jobs processed by a given Process

What if I restarted my notebook and I have no Job ID? The API can ask the process endpoint to list the Jobs for a given Process (deployed Application).


In [ ]:
jobs = api.get_jobs_for_app(app_name)

for job in jobs:
    print("{}/{}".format(job['jobID'], job['status']))

    
print("\n\nFull JSON output data of get_jobs_for_app")
JSON(jobs)